# Grouped Simple Moving Average

This notebook compares Temporian's, pandas' and Polars' version of a grouped simple moving average, aiming to evaluate the speed and ease of writing of each library in this specific use case.

## Data

The data we'll be using belongs to the train CSV of [this Kaggle competition](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=train.csv). It contains daily sales records for grocery stores in Ecuador. Each of the 3 million records corresponds to the daily sales of a certain `family` in a certain store number (`store_nbr`).

To run the notebook, download the CSV and place it in `data/train.csv` in the project's root.

Our goal is to calculate the **weekly moving average** of the sales for **each family** in each **store** (i.e., we want the moving average grouped by those two features).

In [1]:
import numpy as np
import pandas as pd
import temporian as tp

Load data to a pandas DataFrame.

In [2]:
sales_df = pd.read_csv('../data/train.csv', parse_dates=['date'])[['date', 'store_nbr', 'sales', 'family']]
sales_df

,date,store_nbr,sales,family
0,2013-01-01,1,0.000,AUTOMOTIVE
1,2013-01-01,1,0.000,BABY CARE
2,2013-01-01,1,0.000,BEAUTY
3,2013-01-01,1,0.000,BEVERAGES
4,2013-01-01,1,0.000,BOOKS
...,...,...,...,...
3000883,2017-08-15,9,438.133,POULTRY
3000884,2017-08-15,9,154.553,PREPARED FOODS
3000885,2017-08-15,9,2419.729,PRODUCE
3000886,2017-08-15,9,121.000,SCHOOL AND OFFICE SUPPLIES


Load data to a Temporian EventSet.

In [3]:
sales_evset = tp.from_csv('../data/train.csv', timestamps='date')[['sales', 'store_nbr', 'family']]
sales_evset

timestamp,sales,store_nbr,family
2013-01-01 00:00:00+00:00,0,1,AUTOMOTIVE
2013-01-01 00:00:00+00:00,0,1,BABY CARE
2013-01-01 00:00:00+00:00,0,1,BEAUTY
2013-01-01 00:00:00+00:00,0,1,BEVERAGES
2013-01-01 00:00:00+00:00,0,1,BOOKS
…,…,…,…


## Grouped moving average in pandas

We use the `rolling` method of pandas DataFrames to calculate the moving average. We group by `store_nbr` and `family` and then apply the rolling method to the `sales` column.

We'll only measure the time it takes to compute the actual moving average, not the time it takes to group the data, since it is not directly comparable between the two libraries.

In [4]:
grouped_df = sales_df.groupby(['store_nbr', 'family'])

In [5]:
%%time

pd_result = grouped_df.rolling('7d', on='date').mean()

CPU times: user 908 ms, sys: 73.8 ms, total: 981 ms
Wall time: 980 ms


## Grouped moving average in Temporian

Temporian can handle grouped (or hierarchically structured) data natively, using [indexes](https://temporian.readthedocs.io/en/stable/user_guide/#indexes-horizontal-and-vertical-operators). Once our data has the correct index, applying a `simple_moving_average` to it is straightforward.

In [6]:
grouped_evset = sales_evset.add_index(['store_nbr', 'family'])

In [7]:
%%time

tp_result = grouped_evset.simple_moving_average(tp.duration.weeks(1))

CPU times: user 25.6 ms, sys: 490 µs, total: 26.1 ms
Wall time: 26 ms


## Results

Computing the same grouped moving average in Temporian resulted in a **37x speedup** in this dataset!

#### Sanity check

As a sanity check, lets make sure the results from both libraries are the same.

In [8]:
tp_sma = tp.to_pandas(tp_result).sort_values(['store_nbr', 'family', 'timestamp'])['sales']
pd_sma = pd_result.sort_values(['store_nbr', 'family', 'date'])['sales']
np.allclose(pd_sma, tp_sma)

True

## But... what about Polars?

[Polars](https://www.pola.rs/) is a DataFrame library written in Rust, born as a performance-oriented alternative to pandas. Lets see how it fares on this same task!

In [9]:
import polars as pl

polars_df = pl.read_csv("../data/train.csv", try_parse_dates=True, columns=['date', 'store_nbr', 'sales', 'family'])
polars_df.head()

date,store_nbr,family,sales
date,i64,str,f64
2013-01-01,1,"""AUTOMOTIVE""",0.0
2013-01-01,1,"""BABY CARE""",0.0
2013-01-01,1,"""BEAUTY""",0.0
2013-01-01,1,"""BEVERAGES""",0.0
2013-01-01,1,"""BOOKS""",0.0


In [23]:
%%time

pl_result = (
    polars_df
    .rolling(index_column='date', by=['store_nbr', 'family'], period='7d', offset='-7d')
    .agg(pl.col('sales').mean())
)

CPU times: user 824 ms, sys: 172 ms, total: 996 ms
Wall time: 244 ms


### Results

Polars didn't get an edge over pandas - and Temporian is still **37x** faster than it.

Note that this comparison isn't exactly fair though, since Polars doesn't provide a way to apply `.rolling()` over an already grouped DataFrame, which means we're counting both the grouping and the rolling average in the result!

Lets do the same sanity check as before.

In [11]:
pl_sma = pl_result.to_pandas().sort_values(['store_nbr', 'family', 'date'])['sales']
np.allclose(pd_sma, pl_sma)

True

# How does this scale?

3 million records is already a decent amount - but we might just find that scaling it up to larger amounts of data makes Temporian shine even more.

Lets see how each library's performance scales to **5x** and **20x** the original number of rows!

### Pandas

In [12]:
sales_df_5x = pd.concat([sales_df] * 5).sort_values(['date']).reset_index(drop=True)
grouped_df_5x = sales_df_5x.groupby(['store_nbr', 'family'])

sales_df_20x = pd.concat([sales_df] * 20).sort_values(['date']).reset_index(drop=True)
grouped_df_20x = sales_df_20x.groupby(['store_nbr', 'family'])

In [13]:
%%time

_ = grouped_df_5x.rolling('7d', on='date').mean()

CPU times: user 4.9 s, sys: 537 ms, total: 5.44 s
Wall time: 5.44 s


In [14]:
%%time

_ = grouped_df_20x.rolling('7d', on='date').mean()

CPU times: user 21.4 s, sys: 2.4 s, total: 23.8 s
Wall time: 23.8 s


### Polars

In [15]:
polars_df_5x = pl.concat([polars_df] * 5).sort('date')
polars_df_20x = pl.concat([polars_df] * 20).sort('date')

In [16]:
%%time

_ = (
    polars_df_5x
    .rolling(index_column='date', by=['store_nbr', 'family'], period='7d', offset='-7d')
    .agg(pl.col('sales').mean())
)

CPU times: user 6.14 s, sys: 1.72 s, total: 7.86 s
Wall time: 1.1 s


In [18]:
%%time

_ = (
    polars_df_20x
    .rolling(index_column='date', by=['store_nbr', 'family'], period='7d', offset='-7d')
    .agg(pl.col('sales').mean())
)

CPU times: user 47.6 s, sys: 20.3 s, total: 1min 7s
Wall time: 5.68 s


### Temporian

In [17]:
grouped_evset_5x = tp.combine(*([grouped_evset] * 5))  # No need to sort values nor group again - Temporian natively stores data ordered by timestamp and grouped by index!
grouped_evset_20x = tp.combine(*([grouped_evset] * 20))

In [18]:
%%time

_ = grouped_evset_5x.simple_moving_average(tp.duration.weeks(1))

CPU times: user 60.2 ms, sys: 32.1 ms, total: 92.3 ms
Wall time: 91.3 ms


In [19]:
%%time

_ = grouped_evset_20x.simple_moving_average(tp.duration.weeks(1))

CPU times: user 187 ms, sys: 63.9 ms, total: 251 ms
Wall time: 251 ms


# Results

### 5x

Temporian is **60x** faster than pandas and **85x** faster than Polars!

### 20x

Temporian is **95x** faster than pandas and **266x** faster than Polars!